In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 50)
print(os.getcwd())

In [ ]:
df_dismiss = pd.read_csv('DISMISS_Dataset.csv')

In [ ]:
df_dismiss = df_dismiss.rename(columns = {'id_str': 'User_ID'})

In [ ]:
df_dismiss.dtypes

In [ ]:
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_account.loc[df_account.Influencer == 'Influencer']

In [ ]:
df_journalist = pd.read_csv('Additional_Journalist.csv')

In [ ]:
df_journalist['Influencer2'] = 'Influencer'

In [ ]:
df_journalist = df_journalist.drop(columns = 'Influencer')

In [ ]:
df_journalist

In [ ]:
df_account.loc[df_account.Influencer == 'Influencer']

In [ ]:
merged_df = pd.merge(df_account, df_journalist, on='User_ID', how='left')
merged_df['Influencer'] = merged_df['Influencer2'].fillna(merged_df['Influencer'])
merged_df.drop(['Influencer2'], axis=1, inplace=True)
df_account['Influencer'] = merged_df['Influencer']


In [ ]:
df_account.loc[df_account.Influencer == 'Influencer']

In [ ]:
df_cluster = pd.read_csv('Cluster_Tweet_Without_Generic.csv')

In [ ]:
df_cluster.columns

In [ ]:
df_cluster_new = pd.merge(df_cluster, df_account[['User_ID', 'Influencer']]).sort_values(['Cluster_ID', 'Tweet_Time']).drop(columns = ['Unnamed: 0.1','Unnamed: 0']).reset_index(drop = True)

In [ ]:
inf_cluster = df_cluster_new.loc[df_cluster_new.Influencer == 'Influencer'].groupby('Cluster_ID').count().reset_index()['Cluster_ID']

In [ ]:
df_influencer_cluster = df_cluster_new[df_cluster_new.Cluster_ID.isin(inf_cluster)]

In [ ]:
df_influencer_cluster = df_influencer_cluster.reset_index(drop = True)

In [ ]:
influencer_cluster_size = df_influencer_cluster.groupby('Cluster_ID').count().reset_index()[['Cluster_ID', 'Tweet_ID']]

In [ ]:
influencer_cluster_size = influencer_cluster_size.rename(columns = {'Tweet_ID': 'Cluster_Size'})

In [ ]:
influencer_cluster_size

In [ ]:
qw = df_influencer_cluster.loc[df_influencer_cluster.Cluster_ID == 25].reset_index(drop = True)[df_influencer_cluster.loc[df_influencer_cluster.Cluster_ID == 25].reset_index(drop = True).Influencer == 'Influencer'].reset_index()['index'][0]

In [ ]:
influencer_cluster_size['Influencer_Position'] = 0

In [ ]:
for index, row in influencer_cluster_size.iterrows():
    qw = df_influencer_cluster.loc[df_influencer_cluster.Cluster_ID ==  row['Cluster_ID']].reset_index(drop = True)[df_influencer_cluster.loc[df_influencer_cluster.Cluster_ID == row['Cluster_ID']].reset_index(drop = True).Influencer == 'Influencer'].reset_index()['index'][0]
    influencer_cluster_size.at[index, 'Influencer_Position'] = qw + 1


In [ ]:
influencer_cluster_size.loc[influencer_cluster_size.Cluster_Size >= 5]

In [ ]:
influencer_cluster_size.loc[influencer_cluster_size.Influencer_Position == 1]

In [ ]:
influencer_cluster_size.loc[(influencer_cluster_size.Cluster_Size >= 5)&(influencer_cluster_size.Influencer_Position == 1)]

In [ ]:
influencer_cluster_size.loc[influencer_cluster_size.Influencer_Position == 1]

In [ ]:
influencer_cluster_size['Before_Tweet'] = influencer_cluster_size.Influencer_Position - 1
influencer_cluster_size['After_Tweet'] = influencer_cluster_size.Cluster_Size - influencer_cluster_size.Influencer_Position

In [ ]:
influencer_cluster_size.dtypes

In [ ]:
temp = df_influencer_cluster[df_influencer_cluster.Cluster_ID == 25].reset_index(drop = True).iloc[0:1][['Retweet_Count', 'Favorite_Count']]

In [ ]:
sum(temp['Retweet_Count'] + temp['Favorite_Count'])

In [ ]:
for index, row in influencer_cluster_size.iterrows():
    
    temp_before = df_influencer_cluster[df_influencer_cluster.Cluster_ID == row['Cluster_ID']].reset_index(drop = True).iloc[0:row['Before_Tweet'].astype(int)][['Retweet_Count', 'Favorite_Count']]
    influencer_cluster_size.at[index, 'Before_Engagement'] = sum(temp_before['Retweet_Count'] + temp_before['Favorite_Count'])
    
    temp_after = df_influencer_cluster[df_influencer_cluster.Cluster_ID == row['Cluster_ID']].reset_index(drop = True).iloc[row['Influencer_Position'].astype(int):][['Retweet_Count', 'Favorite_Count']]
    influencer_cluster_size.at[index, 'After_Engagement'] = sum(temp_after['Retweet_Count'] + temp_after['Favorite_Count'])
    
    temp_inf = df_influencer_cluster[df_influencer_cluster.Cluster_ID == row['Cluster_ID']].reset_index(drop = True).iloc[row['Influencer_Position'].astype(int)-1][['Retweet_Count', 'Favorite_Count']]
    influencer_cluster_size.at[index, 'Influencer_Engagement'] = temp_inf['Retweet_Count'] + temp_inf['Favorite_Count']



In [ ]:
df_influencer_cluster[df_influencer_cluster.Cluster_ID == 18].reset_index(drop = True).iloc[0:0]

In [ ]:
inf_mid = influencer_cluster_size#[influencer_cluster_size.Influencer_Position != 1]

In [ ]:
inf_mid.Before_Engagement.mean()

In [ ]:
inf_mid.After_Engagement.mean()

In [ ]:
inf_mid.Influencer_Engagement.mean()

In [ ]:
inf_mid

In [ ]:
df_influencer_cluster[df_influencer_cluster.Cluster_ID == 107][['Retweet_Count', 'Favorite_Count']]

In [ ]:
import pandas as pd
from scipy.stats import wilcoxon
import matplotlib.pyplot as plt
statistic, p_value = wilcoxon( inf_mid['After_Engagement'], inf_mid['Before_Engagement'])

In [ ]:
p_value

In [ ]:
statistic

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(inf_mid['Before_Engagement'], bins=10, density=True, cumulative=True, histtype='step', color='blue', label='Before')
plt.hist(inf_mid['After_Engagement'], bins=10, density=True, cumulative=True, histtype='step', color='red', label='After')
plt.xlabel('Engagement')
plt.ylabel('CDF')
plt.title('Cumulative Distribution Function (CDF) of Engagement Before and After Influencer Involvement')
plt.legend()
plt.grid(True)
plt.show()

print("Wilcoxon Test Statistic:", statistic)
print("p-value:", p_value)

if p_value < 0.05:
    print("There is a significant difference in engagement before and after influencer involvement.")
else:
    print("There is no significant difference in engagement before and after influencer involvement.")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

inf_mid_melted = pd.melt(inf_mid, id_vars=['Cluster_ID', 'Cluster_Size', 'Influencer_Position'],
                     value_vars=['Before_Engagement', 'After_Engagement', 'Influencer_Engagement'],
                     var_name='Engagement_Type', value_name='Engagement')

plt.figure(figsize=(10, 6))

sns.scatterplot(data=inf_mid_melted, x='Influencer_Position', y='Engagement', hue='Engagement_Type')


plt.xlabel('Influencer Position')
plt.ylabel('Engagement')
plt.title('Engagement Before and After Influencer Involvement by Influencer Position')
plt.legend(title='Engagement Type')
plt.grid(True)
plt.show()

In [ ]:
summary_statistics_before = inf_mid[['Before_Engagement']].describe()
summary_statistics_after = inf_mid[['After_Engagement']].describe()
summary_statistics_inf = inf_mid[['Influencer_Engagement']].describe()

In [ ]:
summary_statistics_before

In [ ]:
summary_statistics_after

In [ ]:
summary_statistics_inf

In [ ]:
inf_mid[['Before_Tweet']].describe()

In [ ]:
inf_mid[['After_Tweet']].describe()

In [ ]:
df_cluster[df_cluster.Cluster_ID == 167015.0][['User_ID','Tweet_ID','Retweet_Count', 'Favorite_Count']]

In [ ]:
df_influencer_cluster[df_influencer_cluster.Cluster_ID == 167015].reset_index(drop = True).iloc[0:1][['Retweet_Count', 'Favorite_Count']]

In [ ]:
df_account

In [ ]:
df_pol_alignment = pd.read_csv('Political_Alignment_New_Final2.csv')

In [ ]:
df_pol_alignment

In [ ]:
df_account = pd.merge(df_account, df_pol_alignment)

In [ ]:
df_account.loc[df_account.Predicted_PL== 3].groupby('Influencer').count()

In [ ]:
df_account[df_account.RSUOverall>=0.9] 

In [ ]:
df_account.loc[df_account.Influencer == 'Influencer'].groupby('category').count()

In [ ]:
df_account2 = pd.merge(df_account, df_dismiss, on= 'User_ID')

In [ ]:
df_account2.loc[df_account2.Influencer == 'Influencer']

In [ ]:
df_account2.loc[df_account2.Predicted_PL== 0].groupby('sub_category').count()

In [ ]:
df_account3 = df_account2.loc[df_account2.Influencer == 'Influencer']

In [ ]:
df_account3.sub_category = df_account3.sub_category.replace(np.nan, 'journalist')

In [ ]:
df_account3.drop_duplicates('User_ID')

In [ ]:
df_account3.groupby('sub_category').count()

In [ ]:
df_dismiss.loc[df_dismiss.sub_category == 'journalist']

In [ ]:
achik = df_account2.loc[df_account2.sub_category == 'journalist'].User_ID.to_list() + df_journalist.User_ID.tolist()

In [ ]:
df_journalist.User_ID.tolist()

In [ ]:
df_journal = df_account[df_account.User_ID.isin(achik)]

In [ ]:
df_journal[df_journal.Predicted_PL == 3]

In [ ]:
influencer_cluster_size

In [ ]:
df_influencer_cluster.Cluster_ID.tolist()

In [ ]:
df_non_inf_cluster = df_cluster[~df_cluster.Cluster_ID.isin(df_influencer_cluster.Cluster_ID.tolist())]

In [ ]:
df_non_inf_cluster.columns

In [ ]:
df_non_inf_cluster['Total_Engagement'] = df_non_inf_cluster['Retweet_Count'] + df_non_inf_cluster['Favorite_Count']
df_non_inf_cluster['Inf_Camp'] = 'Non-Inf'

In [ ]:
df_influencer_cluster['Total_Engagement'] = df_influencer_cluster['Retweet_Count'] + df_influencer_cluster['Favorite_Count']
df_influencer_cluster['Inf_Camp'] = 'Inf'

In [ ]:
df_non_inf_cluster_engagement = df_non_inf_cluster.groupby('Cluster_ID').sum().reset_index()[['Cluster_ID','Total_Engagement']]

In [ ]:
df_non_inf_cluster_engagement['Inf_Camp'] = 'Non-Inf'

In [ ]:
df_inf_cluster_engagement = df_influencer_cluster[['Cluster_ID', 'Total_Engagement']].groupby('Cluster_ID').sum().reset_index()[['Cluster_ID','Total_Engagement']]

In [ ]:
df_inf_cluster_engagement['Inf_Camp'] = 'Inf'

In [ ]:
df_cluster_size_check = df_cluster.groupby('Cluster_ID').count().reset_index()[['Cluster_ID','Tweet_ID']]

In [ ]:
df_cluster_size_check = df_cluster_size_check.rename(columns = {'Tweet_ID': 'Total_Tweet'})

In [ ]:
df_non_inf_cluster = pd.merge(df_non_inf_cluster, df_account[['User_ID', 'verification']])

In [ ]:
df_non_inf_cluster.loc[df_non_inf_cluster.verification == 'Verified'].Cluster_ID.tolist()

In [ ]:
df_engagement_stat = pd.concat([df_inf_cluster_engagement, df_non_inf_cluster_engagement])

In [ ]:
df_engagement_stat = pd.merge(df_engagement_stat, df_cluster_size_check)

In [ ]:
df_engagement_stat_without_ver = df_engagement_stat[~df_engagement_stat.Cluster_ID.isin(df_non_inf_cluster.loc
                                                      [df_non_inf_cluster.verification == 'Verified'].
                                                      Cluster_ID.tolist())]

In [ ]:
df_engagement_stat_without_ver

In [ ]:
df_engagement_stat_without_ver['Avg_Engagement'] = df_engagement_stat_without_ver['Total_Engagement']/df_engagement_stat_without_ver['Total_Tweet']

In [ ]:
df_engagement_stat_without_ver.loc[df_engagement_stat_without_ver.Cluster_ID == 34]

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind


print(df_engagement_stat_without_ver.head())

engagement_with_influencers = df_engagement_stat[df_engagement_stat['Inf_Camp'] == 'Inf']['Avg_Engagement']
engagement_without_influencers = df_engagement_stat[df_engagement_stat['Inf_Camp'] == 'Non-Inf']['Avg_Engagement']

t_statistic, p_value = ttest_ind(engagement_with_influencers, engagement_without_influencers)

print("T-Statistic:", t_statistic)
print("P-Value:", p_value)

alpha = 0.05
if p_value < alpha:
    print("There is a statistically significant difference in engagement between campaigns with influencers and campaigns without influencers.")
else:
    print("There is no statistically significant difference in engagement between campaigns with influencers and campaigns without influencers.")


In [ ]:
engagement_with_influencers.describe()

In [ ]:
engagement_without_influencers.describe()

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(x='Inf_Camp', y='Avg_Engagement', data=df_engagement_stat, showfliers=False)
plt.xlabel('Campaign Type')
plt.ylabel('Average Engagement')
plt.title('Total Engagement Distribution for Campaigns with and without Influencers')
plt.xticks([0, 1])
plt.show()

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(x='Inf_Camp', y='Avg_Engagement', data=df_engagement_stat, showfliers=False)
plt.xlabel('Campaign Type')
plt.ylabel('Average Engagement')
plt.title('Total Engagement Distribution for Campaigns with and without Influencers')
plt.xticks([0, 1])
plt.show()

In [ ]:
df_engagement_stat.loc[df_engagement_stat.Cluster_ID == 34]

In [ ]:
df_pol_acc = pd.read_csv('Political_Alignment_New_Final2.csv')

In [ ]:
df_account_up = pd.merge(df_account, df_pol_acc)

In [ ]:
df_account_up[['User_ID','Predicted_PL']][df_account_up[['User_ID','Predicted_PL']].Predicted_PL == 3]

In [ ]:
pd.merge(df_cluster, df_account_up[['User_ID','Predicted_PL']][df_account_up[['User_ID','Predicted_PL']].Predicted_PL == 3]).groupby('Cluster_ID').count().reset_index()[['Cluster_ID', 'Tweet_ID']]

In [ ]:
df_engagement_stat = pd.merge(df_engagement_stat, pd.merge(df_cluster, df_account_up[['User_ID','Predicted_PL']][df_account_up[['User_ID','Predicted_PL']].Predicted_PL == 3]).groupby('Cluster_ID').count().reset_index()[['Cluster_ID', 'Tweet_ID']], how = 'left')

In [ ]:
df_engagement_stat.Tweet_ID = df_engagement_stat.Tweet_ID.replace(np.nan, 0)

In [ ]:
df_engagement_stat = df_engagement_stat.rename(columns = {'Tweet_ID': 'Non_Pol_Acc_Number'})

In [ ]:
df_engagement_stat_new = df_engagement_stat.loc[df_engagement_stat.Non_Pol_Acc_Number > 0]

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind


print(df_engagement_stat_new.head())

engagement_with_influencers = df_engagement_stat_new[df_engagement_stat_new['Inf_Camp'] == 'Inf']['Non_Pol_Acc_Number']
engagement_without_influencers = df_engagement_stat_new[df_engagement_stat_new['Inf_Camp'] == 'Non-Inf']['Non_Pol_Acc_Number']

t_statistic, p_value = ttest_ind(engagement_with_influencers, engagement_without_influencers)

print("T-Statistic:", t_statistic)
print("P-Value:", p_value)

alpha = 0.05
if p_value < alpha:
    print("There is a statistically significant difference in engagement between campaigns with influencers and campaigns without influencers.")
else:
    print("There is no statistically significant difference in engagement between campaigns with influencers and campaigns without influencers.")


In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(x='Inf_Camp', y='Non_Pol_Acc_Number', data=df_engagement_stat_new, showfliers=False)
plt.xlabel('Campaign Type')
plt.ylabel('Number of Non-pol acc')
plt.title('Total Engagement Distribution for Campaigns with and without Influencers')
plt.xticks([0, 1])
plt.show()

In [ ]:
engagement_with_influencers.describe()

In [ ]:
engagement_without_influencers.describe()

In [ ]:
t_statistic_1 = 4.479174165411675
p_value_1 = 7.764040518789608e-06
t_statistic_2 = 75.21786628924215
p_value_2 = 0.0

num_tests = 2

alpha = 0.05

adjusted_alpha = alpha / num_tests

adjusted_p_value_1 = p_value_1 * num_tests
adjusted_p_value_2 = p_value_2 * num_tests

if adjusted_p_value_1 < adjusted_alpha:
    print("Test 1 is statistically significant after Bonferroni correction.")
else:
    print("Test 1 is not statistically significant after Bonferroni correction.")

if adjusted_p_value_2 < adjusted_alpha:
    print("Test 2 is statistically significant after Bonferroni correction.")
else:
    print("Test 2 is not statistically significant after Bonferroni correction.")


In [ ]:
adjusted_p_value_1

In [ ]:
adjusted_p_value_2

In [ ]:
df_engagement_stat.loc[df_engagement_stat.Total_Tweet <= 3]

In [ ]:
df_cluster_old = pd.read_csv("Cluster_Tweet_Final.csv")

In [ ]:
df_cluster_old.groupby('Cluster_ID').count().reset_index().sort_values('Tweet_ID')

In [ ]:
df_cluster.groupby('Cluster_ID').count().reset_index().sort_values('Tweet_ID')

In [ ]:
df_cluster.loc[df_cluster.Cluster_ID == 129331].User_ID.tolist()

In [ ]:
df_account[df_account.User_ID.isin(df_cluster.loc[df_cluster.Cluster_ID == 154899].User_ID.tolist())]

In [ ]:
df_cluster.loc[df_cluster.Cluster_ID == 154899][['Retweet_Count', 'Favorite_Count']].sum().sum()

In [ ]:
len(df_cluster.loc[df_cluster.Cluster_ID == 154899])

In [ ]:
df_cluster.loc[df_cluster.Cluster_ID == 154899].Tweet_Text.sample(1)

In [ ]:
df_account[(df_account.acc_type == 'Bot') & (df_account.Predicted_PL == 3)]

In [ ]:
df_all_tweet[df_all_tweet.User_ID.isin(df_account[(df_account.acc_type == 'Bot') & (df_account.Predicted_PL == 3)].User_ID.tolist())].sample(20)

In [ ]:
df_group = pd.read_csv('Refined_Community_with_Political_Label2.csv')

In [ ]:
df_group.User_ID

In [ ]:
df_account_group = pd.merge(df_account, df_group)

In [ ]:
grp = [0,18,8,1,17,6,15,19,13,4,29,14,42,5]

In [ ]:
for i in grp:
    print('Group:' + str(i))
    print(len(df_account_group[(df_account_group.Community_ID == i) & 
                     (df_account_group.Influencer == 'Influencer')]))


In [ ]:
cl_s = df_cluster.groupby('Cluster_ID').count().reset_index()

In [ ]:
cl_s[cl_s.Tweet_ID >= 5]

In [ ]:
df_cluster

In [ ]:
df_account.groupby('Influencer').count()

In [ ]:
df_account.to_csv('All_Account_with_Updated_Influencer.csv', index = False)

In [ ]:
df_cluster_new = pd.merge(df_cluster[['Cluster_ID', 'Tweet_ID', 'User_ID']], df_account, on = 'User_ID')

In [ ]:
df_cluster_new[df_cluster_new.acc_type == 'Bot'].groupby('Cluster_ID').count()

In [ ]:
df_cluster_new[(df_cluster_new.Predicted_PL == 0) | 
               (df_cluster_new.Predicted_PL == 1) |
              (df_cluster_new.Predicted_PL == 2)].groupby('Cluster_ID').count()

In [ ]:
df_cluster_new.groupby('Cluster_ID').count()